In [1]:
import pandas as pd
import numpy as np

In [3]:
size = 10000
user_chunks = pd.read_json('../data/yelp_academic_dataset_user.json', lines=True,
                    dtype = {
                        "review_count": np.int16,
                        "useful": np.int16,
                        "funny": np.int16,
                        "cool": np.int16,
                        "fans": np.int16,
                        "average_stars": np.float32, # calculations are done on float32 anyways for float16
                        "compliment_hot": np.int16,
                        "compliment_more": np.int16,
                        "compliment_profile": np.int16,
                        "compliment_cute": np.int16,
                        "compliment_list": np.int16,
                        "compliment_note": np.int16,
                        "compliment_plain": np.int16,
                        "compliment_cool": np.int16,
                        "compliment_funny": np.int16,
                        "compliment_writer": np.int16,
                        "compliment_photos": np.int16
                    }, chunksize=size)

In [4]:
chunk_list = []
for chunk in user_chunks:
    # only keep tips of current or former elite users.
    # chunk = chunk[chunk.user_id.isin(elite_user_ids)]
    
    chunk_list.append(chunk)
    break
user = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)

In [19]:
user.iloc[343]

user_id                                          hizGc5W1tBHPghM5YKCAtg
name                                                              Katie
review_count                                                       1754
yelping_since                                       2009-05-22 20:19:55
useful                                                           -19002
funny                                                             31112
cool                                                             -27572
elite                 2009,2010,2011,2012,2013,2014,2015,2016,2017,2...
friends               U_U0yg03tOYHQlnCQhZAZg, ffarmP6gowdXoTxQARKU7A...
fans                                                               3511
average_stars                                                      4.23
compliment_hot                                                     6560
compliment_more                                                     424
compliment_profile                                              

In [18]:
user.friends.map(len).sort_values()

8024        22
8273        22
8330        22
1701        22
6221        22
         ...  
530     177670
5524    179206
2516    192886
305     210670
343     214366
Name: friends, Length: 10000, dtype: int64

# Read businesses
Keep only true restaurants

In [15]:
size = 100000
business_chunks = pd.read_json('../data/yelp_academic_dataset_business.json', lines=True,
                    dtype={
                        "business_id": str,
                        "name": str,
                        "address": str,
                        "city": str,
                        "state": str,
                        "postal code": str,
                        "latitude": float,
                        "longitude": float,
                        "stars": float,
                        "review_count": int,
                        "is_open": int
                    }, chunksize=size)

chunk_list = []
for chunk in business_chunks:
    chunk = chunk[chunk['categories'].notnull()]
    
    # only keep restaurants
    chunk = chunk.loc[[('Restaurant' in x) for x in chunk['categories']]]
    
    # only certain columns
    # chunk = chunk[['business_id', 'name','stars', 'is_open','review_count']]
    
    chunk_list.append(chunk)

restaurants = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)


In [16]:
restaurants.head()

business_id                     name              address  \
0  6iYb2HFDywm3zjuRg0shjw      Oskar Blues Taproom         921 Pearl St   
1  tCbdrRPZA0oiIYSmHG3J0w  Flying Elephants at PDX  7000 NE Airport Way   
2  D4JtQNTI4X3KcbzacDJsMw      Bob Likes Thai Food         3755 Main St   
3  jFYIsSb7r1QeESVUnXPHBw          Boxwood Biscuit        740 S High St   
4  HPA_qyMEddpAEtFof02ixg      Mr G's Pizza & Subs        474 Lowell St   

        city state postal_code   latitude   longitude  stars  review_count  \
0    Boulder    CO       80302  40.017544 -105.283348    4.0            86   
1   Portland    OR       97218  45.588906 -122.593331    4.0           126   
2  Vancouver    BC         V5V  49.251342 -123.101333    3.5           169   
3   Columbus    OH       43206  39.947007  -82.997471    4.5            11   
4    Peabody    MA       01960  42.541155  -70.973438    4.0            39   

   is_open                                         attributes  \
0        1  {'RestaurantsTableService': 'True', 'WiFi': 'u...   
1        1  {'RestaurantsTakeOut': 'True', 'RestaurantsAtt...   
2        1  {'GoodForKids': 'True', 'Alcohol': 'u'none'', ...   
3        1                                               None   
4        1  {'RestaurantsGoodForGroups': 'True', 'HasTV': ...   

                                          categories  \
0  Gastropubs, Food, Beer Gardens, Restaurants, B...   
1  Salad, Soup, Sandwiches, Delis, Restaurants, C...   
2                                  Restaurants, Thai   
3                    Breakfast & Brunch, Restaurants   
4                           Food, Pizza, Restaurants   

                                               hours  
0  {'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...  
1  {'Monday': '5:0-18:0', 'Tuesday': '5:0-17:0', ...  
2  {'Monday': '17:0-21:0', 'Tuesday': '17:0-21:0'...  
3     {'Saturday': '8:0-14:0', 'Sunday': '8:0-14:0'}  
4  {'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'...

In [17]:
restaurants.shape

(50793, 14)

In [29]:
restaurants.to_pickle('../data/restaurants.pkl')

In [23]:
attributes = list(set([attribute for attributes in restaurants.attributes.values  if attributes is not None for attribute in attributes]))

# Read Reviews

In [25]:
restaurant_ids = restaurants.business_id.values

In [26]:
size = 1000000
review_chunks = pd.read_json('../data/yelp_academic_dataset_review.json', lines=True,
                    dtype={
                    "review_id": str,
                    "user_id": str,
                    "business_id": str,
                    "stars": int,
                    "date": str,
                    "text": str,
                    "useful": int,
                    "funny": int,
                    "cool": int,
                }, chunksize=size)

chunk_list = []
for chunk in review_chunks:
    # only keep tips of current or former elite users.
    chunk = chunk[chunk.business_id.isin(restaurant_ids)]
    chunk_list.append(chunk)

reviews = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)

In [27]:
reviews.shape

(5575349, 9)

In [30]:
reviews.to_pickle('../data/restaurant_reviews.pkl')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\yelp\lib\site-packages\IPython\core\interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\lukas\AppData\Local\Temp/ipykernel_6496/2807565378.py", line 1, in <module>
    reviews.to_pickle('../data/restaurant_reviews.pkl')
  File "C:\ProgramData\Anaconda3\envs\yelp\lib\site-packages\pandas\core\generic.py", line 2957, in to_pickle
    to_pickle(
  File "C:\ProgramData\Anaconda3\envs\yelp\lib\site-packages\pandas\io\pickle.py", line 113, in to_pickle
    pickle.dump(
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\yelp\lib\site-packages\IPython\core\interactiveshell.py", line 2064, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above 

TypeError: object of type 'NoneType' has no len()

Preprocess text of reviews.
We want to:
* stem words
* remove extra whitespaces
* lower case

In [31]:
# Load the regular expression library
import re
# Remove punctuation
reviews['text_processed'] = \
reviews['text'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
reviews['text_processed'] = \
reviews['text_processed'].map(lambda x: x.lower())
# Print out the first rows of papers
reviews['text_processed'].head()

0    apparently prides osteria had a rough summer a...
1    i've stayed at many marriott and renaissance m...
2    the food is always great here the service from...
3    this place used to be a cool chill place now i...
4    the setting is perfectly adequate and the food...
Name: text_processed, dtype: object

In [32]:
reviews.shape

(5575349, 10)

## Wordcloud
Check the word distributions of the texts.

In [ ]:
# Import the wordcloud library
from wordcloud import WordCloud
# Join the different processed titles together.
long_string = ','.join(list(reviews['text_processed'].values))
# Create a WordCloud object
wordcloud = WordCloud(background_color="white", max_words=5000, contour_width=3, contour_color='steelblue')
# Generate a word cloud
wordcloud.generate(long_string)
# Visualize the word cloud
wordcloud.to_image()